In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import csv
import time
import requests, bs4

In [189]:
def get_links(url):
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    #table = soup.find_all("tr", {"class": "no-border tm-user-tendenz"}).find_all('td')
    table = soup.find_all("tr", {"class": "no-border tm-user-tendenz"})
    table_teams = soup.find_all("tr", {"class": "table-grosse-schrift"})
    #print(table_teams)
    probs = dict()
    for i in range(0,10):
        table_tmp = table[i].find_all('td')
        table_teams_tmp =  table_teams[i].find_all('td')
        if 'home_team' in probs:
            probs ['home_team'].append (table_teams_tmp[0].contents[3].contents[0])
        else:
            probs ['home_team'] = [table_teams_tmp[0].contents[3].contents[0]]
        if 'away_team' in probs:
            probs ['away_team'].append (table_teams_tmp[7].contents[1].contents[0])
        else:
            probs ['away_team'] = [table_teams_tmp[7].contents[1].contents[0]]
        if 'home_prob' in probs:
            probs ['home_prob'].append (table_tmp[0].contents[1].contents[0].contents[0])
        else:
            probs ['home_prob'] = [table_tmp[0].contents[1].contents[0].contents[0]]
        if 'draw_prob' in probs:
            probs ['draw_prob'].append (table_tmp[1].contents[1].contents[0].contents[0])
        else:
            probs ['draw_prob'] = [table_tmp[1].contents[1].contents[0].contents[0]]
        if 'away_prob' in probs:
            probs ['away_prob'].append (table_tmp[2].contents[1].contents[0].contents[0])
        else:
            probs ['away_prob'] = [table_tmp[2].contents[1].contents[0].contents[0]]

    #df_player = pd.DataFrame.from_dict(pre_df_player)

    out = pd.DataFrame.from_dict(probs)

    return  out

test = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag=4')
#get_links('https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures')
test

,home_team,away_team,home_prob,draw_prob,away_prob
0,Chelsea,Crystal Palace,93.9 %,4.1 %,2.0 %
1,Everton,Brighton,95.5 %,4.5 %,%
2,Leeds,Man City,6.3 %,13.7 %,80.0 %
3,Newcastle,Burnley,84.0 %,12.1 %,3.9 %
4,Leicester,West Ham,97.1 %,2.4 %,0.5 %
5,Southampton,West Brom,65.7 %,26.6 %,7.7 %
6,Arsenal,Sheff Utd,97.1 %,2.4 %,0.5 %
7,Wolves,Fulham,84.6 %,10.6 %,4.8 %
8,Man Utd,Spurs,38.5 %,43.8 %,17.8 %
9,Aston Villa,Liverpool,1.4 %,1.9 %,96.6 %


In [193]:
stri = 'https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag='
df = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag=4')
for i in range (5, 39):
    stri_tmp = stri
    word ="{}".format(i)
    stri_tmp += word
    df_tmp = get_links(stri_tmp)
    df = pd.concat([df, df_tmp])

df

,home_team,away_team,home_prob,draw_prob,away_prob
0,Chelsea,Crystal Palace,93.9 %,4.1 %,2.0 %
1,Everton,Brighton,95.5 %,4.5 %,%
2,Leeds,Man City,6.3 %,13.7 %,80.0 %
3,Newcastle,Burnley,84.0 %,12.1 %,3.9 %
4,Leicester,West Ham,97.1 %,2.4 %,0.5 %
...,...,...,...,...,...
5,Liverpool,Crystal Palace,96.2 %,2.5 %,1.3 %
6,Man City,Everton,83.4 %,10.2 %,6.4 %
7,Sheff Utd,Burnley,17.2 %,24.2 %,58.6 %
8,West Ham,Southampton,87.9 %,8.9 %,3.2 %


In [194]:
df.replace(['Man Utd', 'Sheff Utd', 'Spurs'], ['Man United','Sheffield United', 'Tottenham'], inplace=True)
df['home_prob'] = pd.to_numeric(df['home_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
df['draw_prob'] = pd.to_numeric(df['draw_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
df['away_prob'] = pd.to_numeric(df['away_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
df = df.fillna(0)
df['game_id'] = df['home_team'] + df['away_team']
df

,home_team,away_team,home_prob,draw_prob,away_prob,game_id
0,Chelsea,Crystal Palace,0.939,0.041,0.020,ChelseaCrystal Palace
1,Everton,Brighton,0.955,0.045,0.000,EvertonBrighton
2,Leeds,Man City,0.063,0.137,0.800,LeedsMan City
3,Newcastle,Burnley,0.840,0.121,0.039,NewcastleBurnley
4,Leicester,West Ham,0.971,0.024,0.005,LeicesterWest Ham
...,...,...,...,...,...,...
5,Liverpool,Crystal Palace,0.962,0.025,0.013,LiverpoolCrystal Palace
6,Man City,Everton,0.834,0.102,0.064,Man CityEverton
7,Sheffield United,Burnley,0.172,0.242,0.586,Sheffield UnitedBurnley
8,West Ham,Southampton,0.879,0.089,0.032,West HamSouthampton


In [197]:
df = df.rename(columns={'home_prob':'H','draw_prob':'D','away_prob':'A'})
df

,home_team,away_team,H,D,A,game_id
0,Chelsea,Crystal Palace,0.939,0.041,0.020,ChelseaCrystal Palace
1,Everton,Brighton,0.955,0.045,0.000,EvertonBrighton
2,Leeds,Man City,0.063,0.137,0.800,LeedsMan City
3,Newcastle,Burnley,0.840,0.121,0.039,NewcastleBurnley
4,Leicester,West Ham,0.971,0.024,0.005,LeicesterWest Ham
...,...,...,...,...,...,...
5,Liverpool,Crystal Palace,0.962,0.025,0.013,LiverpoolCrystal Palace
6,Man City,Everton,0.834,0.102,0.064,Man CityEverton
7,Sheffield United,Burnley,0.172,0.242,0.586,Sheffield UnitedBurnley
8,West Ham,Southampton,0.879,0.089,0.032,West HamSouthampton


In [198]:
df.to_csv('Data/2020-21/Fixtures/tranfermarkt_probs.csv')